This code uses the SQLite version of the Standard Works of the Church of Jesus Christ of Latter-Day Saints, available [here](https://scriptures.nephi.org/)

In [ ]:
;;;    select-scripture
;;;    Get verses of scripture by reference.
;;;    Copyright (C) 2020  Jesse Gibbons
;;;
;;;    This program is free software: you can redistribute it and/or modify
;;;    it under the terms of the GNU General Public License as published by
;;;    the Free Software Foundation, either version 3 of the License, or
;;;    (at your option) any later version.
;;;
;;;    This program is distributed in the hope that it will be useful,
;;;    but WITHOUT ANY WARRANTY; without even the implied warranty of
;;;    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
;;;    GNU General Public License for more details.
;;;
;;;    You should have received a copy of the GNU General Public License
;;;    along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Database Access Functions:

In [ ]:



(use-modules (sqlite3)
             (srfi srfi-1))

(define database-file "lds-scriptures-sqlite.db")

(define (run-simple-statement-string db str)
  (let*
    ((stmt (sqlite-prepare db str))
     (results (sqlite-map identity stmt)))
    (sqlite-finalize stmt)
    results))

(define (table-column-names db table-name)
  (run-simple-statement-string db (string-append "SELECT * FROM " table-name)))

(define (all-table-names db)
  (run-simple-statement-string db "SELECT name FROM sqlite_master"))

(define (book-names db)
  (reduce-right append '()
                (map vector->list
                     (run-simple-statement-string db "SELECT book_title, upper(book_title), lower(book_title),
                                                  book_short_title, upper(book_short_title), lower(book_short_title),
                                                  book_lds_url, upper(book_lds_url), lower(book_lds_url)
                                                  FROM books"))))

# Initial Database Access

In [ ]:
(define book-list
  (let* ((db (sqlite-open database-file))
        (result (cons 'or (delete-duplicates (book-names db)))))
    (sqlite-close db)
    result))

# Parse Tree Generator functions

In [ ]:
(use-modules (ice-9 peg))
(use-modules (ice-9 pretty-print))

(define-peg-string-patterns
 "number <-- [0-9]+
range <-- (number '-' number) / (number)
ranges <- (range ',' (' ')? ranges) / range
chapter-ref <-- number (':' ranges) ?
chapters <-- (chapter-ref ';' (' ' ?) chapters) / chapter-ref")

(define book-title (compile (compile-peg-pattern book-list 'all)))

(define-peg-string-patterns
 "space <- ' '
book-reference <-- book-title space chapters
scripture-reference <-- (book-reference (space / '\n') scripture-reference) / book-reference")

# Processing Functions
## Number

In [ ]:
(define (process-number number)
  (string->number (cadr number)))

## Verses

In [ ]:
(define (process-range range)
  (let* ((numbers (filter list? (keyword-flatten '(number) range)))
        (parsed-numbers (map process-number numbers)))
    (case (length parsed-numbers)
          ((1) parsed-numbers)
          ((2) (iota (- (1+ (second parsed-numbers)) (first parsed-numbers)) (first parsed-numbers)))
          (else (raise 'parse-error)))))
(define (process-ranges ranges)
  (reduce-right append '() (map process-range ranges)))

## Chapter

In [ ]:
(define (chapter-ref-number chapter-ref)
  (process-number (first (filter list? (keyword-flatten 
                                        '(number) chapter-ref)))))

(define (chapter-verse-ranges chapter-ref)
  (filter list? (keyword-flatten '(range) chapter-ref)))

(define (entire-chapter? chapter-ref)
    (null-list? (chapter-verse-ranges chapter-ref)))

(define (process-chapter-ref chapter-ref)
  (let* ((whole-chapter? (entire-chapter? chapter-ref))
         (chapter-number (chapter-ref-number chapter-ref)))
    (if whole-chapter? (list whole-chapter? chapter-number)
        (list whole-chapter? chapter-number
              (process-ranges (chapter-verse-ranges chapter-ref))))))

## Book

In [ ]:
(define (book-ref-book book-reference)
  (string-trim-both (first (filter string? book-reference))))
(define (book-ref-chapters book-ref)
  (filter list? (keyword-flatten '(chapter-ref) book-ref)))

(define (process-book-ref book-ref)
  (let ((book-name (book-ref-book book-ref))
        (chapters (book-ref-chapters book-ref)))
    (cons book-name (map (lambda (chapter-tree)
                           (process-chapter-ref chapter-tree))
         chapters))))

(define (clean-reference-tree peg)
  (filter list? (keyword-flatten '(book-reference) (peg:tree peg))))

(define (process-reference-tree reference-tree)
  (map process-book-ref reference-tree))

## Lookup

In [ ]:
(define select-verses-string "SELECT scripture_text FROM verses
WHERE verse_number = :verse_number
AND chapter_id IN
(SELECT id FROM chapters
WHERE chapter_number = :chapter_number
AND book_id IN 
(SELECT id FROM books
WHERE book_title LIKE :book_title
OR book_short_title LIKE :book_title
OR book_lds_url LIKE :book_title))")
(define select-chapter-string "SELECT scripture_text FROM verses
WHERE chapter_id IN
(SELECT id FROM chapters
WHERE chapter_number = :chapter_number
AND book_id IN 
(SELECT id FROM books
WHERE book_title LIKE :book_title
OR book_short_title LIKE :book_title
OR book_lds_url LIKE :book_title))")
(define (collect-verses stmt)
  (let ((result
         (sqlite-map
          (lambda (row)
            (vector-ref row 0))
          stmt)))
    (sqlite-reset stmt)
    result))


(define (lookup-references processed-refs)
  (let* ((db (sqlite-open database-file))
         (select-verses-stmt
           (sqlite-prepare db select-verses-string))
         (select-chapter-stmt
           (sqlite-prepare db select-chapter-string))
         (result
           (map (lambda (ref)
                  (let ((book-title (first ref)))
                    (sqlite-bind
                      select-verses-stmt
                      ":book_title"
                      (first ref))
                    (sqlite-bind
                      select-chapter-stmt
                      ":book_title"
                      (first ref))
                    (reduce-right
                      append
                      '()
                      (map (lambda (selection)
                             (if (first selection)
                               (let ((stmt select-chapter-stmt)
                                     (chapter-number (second selection)))
                                 (sqlite-bind
                                   stmt
                                   ":chapter_number"
                                   chapter-number)
                                 (collect-verses stmt))
                               (let ((stmt select-verses-stmt)
                                     (chapter-number (second selection)))
                                 (sqlite-bind
                                   stmt
                                   ":chapter_number"
                                   chapter-number)
                                       (reduce-right append '() (map (lambda (verse)
                                        (sqlite-bind
                                          stmt
                                          ":verse_number"
                                          verse)
                                        (collect-verses stmt))
                                      (third selection))))))
                           (cdr ref)))))
                processed-refs)))
    (sqlite-finalize select-verses-stmt)
    (sqlite-finalize select-chapter-stmt)
    (sqlite-close db)
    (reduce-right append '() result)))

(define (lookup ref-string)
  (lookup-references
   (process-reference-tree
    (clean-reference-tree
     (match-pattern scripture-reference ref-string)))))

In [ ]:
(display (let ((reference (apply string-append (map (lambda (str) (string-append str " ")) (cdr (command-line))))))
   (reduce-right string-append (string-append reference ":") (lookup reference))))